In [1]:
%load_ext autoreload
%autoreload 2

import sys, glob, os, gzip
sys.path.insert(1, os.path.join(sys.path[0], '..'))

import numpy as np
import pandas as pd

import torch
import pytorch_lightning as pl
from pytorch_lightning.loggers.neptune import NeptuneLogger

from ablang_train import ABtokenizer, AbLang, trainingframe
from ablang_train.train_utils import callback_handler, datamodule, arghandler

In [2]:
def set_neptune_logger(args):
    """
    Initialize Neptune logger
    """

    neptune_args = { 'api_key':"eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI0N2Y2YmIxMS02OWM3LTRhY2MtYTQxOC0xODU5N2E0ODFmMzEifQ==",
    'project':"tobiasheol/AbLangTraining",
    'name':args.name,
    'log_model_checkpoints':False,
    }

    return NeptuneLogger(**neptune_args)

In [3]:
# SET ARGUMENTS AND HPARAMS
arguments = arghandler.parse_args(args="", is_test=True)
logger = set_neptune_logger(arguments.model_specific_args)

arguments.model_specific_args.num_encoder_blocks = 1
arguments.model_specific_args.over_sample_data = 0
arguments.model_specific_args.data_path = '../data/single_data/'
arguments.model_specific_args.max_fit_batch_size = 1
arguments.model_specific_args.effective_batch_size = 1
arguments.model_specific_args.eval_batch_size = 1
arguments.model_specific_args.val_check_interval = 1
arguments.model_specific_args.use_tkn_dropout = False

arguments.trainer_args['logger'] = logger
arguments.trainer_args['log_every_n_steps'] = 1

callbacks = callback_handler.CallbackHandler(save_step_frequency=1, 
                                progress_refresh_rate=0, 
                                outpath=arguments.model_specific_args.out_path)

In [4]:
def enforce_reproducibility(seed=42):
    # Sets seed manually for both CPU and CUDA
    torch.manual_seed(seed)
    # For atomic operations there is currently
    # no simple way to enforce determinism, as
    # the order of parallel operations is not known.
    #
    # CUDNN
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # System based
    np.random.seed(seed)
    
    pl.seed_everything(seed)
    
# SET SEED - IMPORTANT FOR MULTIPLE GPUS, OTHERWISE GOOD FOR REPRODUCIBILITY
enforce_reproducibility(arguments.model_specific_args.seed)

Global seed set to 42


In [9]:
# LOAD AND INITIATE DATA
arguments.model_specific_args.mask_technique = 'random'

abrep_dm = datamodule.MyDataModule(arguments.model_specific_args, ABtokenizer) 
# You are supposed to just be able to add abrep to the fit function, but it doesn't work when using multiple GPUs
abrep_dm.setup('fit')

train = abrep_dm.train_dataloader()
val = abrep_dm.val_dataloader()

for batch in train:
    
    print(batch['input'][0])
    print(batch['labels'][:len(batch['input'][0])])
    break

tensor([ 0, 10,  7, 23, 20,  8, 10, 13, 13,  7, 20,  7,  6, 14, 13,  2, 23,  2,
         3,  8, 16,  7, 11,  7, 12,  7, 23,  7,  9, 16, 23,  9, 23, 23, 23,  9,
        10, 23,  2, 10, 23, 23, 12,  4,  8, 13, 16, 20, 23, 16, 23, 18,  5, 23,
        20, 23, 13, 23, 12, 23,  7,  5,  2, 23,  7, 23,  7,  4,  7, 23,  8,  7,
        14, 23, 23, 14, 16,  7, 12, 20, 10,  7,  6,  5,  6, 14,  5, 18, 18, 11,
        14,  8, 19,  5,  5,  7, 23,  9, 12, 23, 23, 17, 23, 12, 12,  8,  4, 20,
         8, 15, 20, 22, 22, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21,
        21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21,
        21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21,
        21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21,
        21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21,
        21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21,
        21, 21, 21, 21, 21, 21, 21])
ten

In [6]:
# LOAD MODEL
model = trainingframe.TrainingFrame(arguments.model_specific_args, AbLang, ABtokenizer)

# INITIALISE TRAINER
trainer = pl.Trainer(**arguments.trainer_args, callbacks=callbacks())

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1)` was configured so validation will run after every batch.


In [7]:
trainer.fit(model, train, val)


  | Name    | Type             | Params
---------------------------------------------
0 | loss_fn | CrossEntropyLoss | 0     
1 | ablang  | AbLang           | 7.7 M 
---------------------------------------------
7.7 M     Trainable params
0         Non-trainable params
7.7 M     Total params
30.800    Total estimated model params size (MB)


https://app.neptune.ai/tobiasheol/AbLangTraining/e/ABLANG-319


Info (NVML): NVML Shared Library Not Found. GPU usage metrics may not be reported. For more information, see https://docs.neptune.ai/you-should-know/what-can-you-log-and-display#hardware-consumption


Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


/data/iraqbabbler/olsen/miniconda3/envs/ablang-train/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/data/iraqbabbler/olsen/miniconda3/envs/ablang-train/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


['CHECKPOINT_HYPER_PARAMS_KEY', 'CHECKPOINT_HYPER_PARAMS_NAME', 'CHECKPOINT_HYPER_PARAMS_TYPE', 'T_destination', '_DeviceDtypeModuleMixin__update_properties', '_LightningModule__check_allowed', '_LightningModule__check_not_nested', '_LightningModule__to_tensor', '__annotations__', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__jit_unused_properties__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_apply', '_apply_batch_transfer_handler', '_auto_collect_arguments', '_automatic_optimization', '_backward_hooks', '_buffers', '_call_impl', '_current_fx_name', '_device', '_dtype', '_example_input_array', '_forward_hooks', '_forward_pre_hooks', '_get_backward_hooks', '_get_name', '_h

/data/iraqbabbler/olsen/miniconda3/envs/ablang-train/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:726: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
